In [1]:
import os 
import sys 
import json 
from functools import cache 
from collections import OrderedDict, defaultdict
from pathlib import Path 
from functools import cache
from itertools import product

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

load_dotenv('../../..')

from utils_notebook.testing import validate_season_series
from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys, dict_invert
from utils_notebook.constants import ADDRS, ADDRS_SILO_TOKENS, TOKEN_NAMES_SILO, DECIMALS_SILO_TOKENS
from utils_notebook.vega import output_chart

In [2]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

## Siloed Asset Breakdown 

In [3]:
# See UI logic in this directory for reference 
# https://github.com/BeanstalkFarms/Beanstalk-UI/tree/master/src/components/Analytics/Silo
asset_ids = [
    f"{ADDRS.beanstalk}-{token_addr}" for token_addr in ADDRS_SILO_TOKENS.values()
]
token_name_map = dict_invert(ADDRS_SILO_TOKENS)

@cache 
def query_silo_asset_hourly_snapshots(refresh=None): 
    queries = [
        bs.Query.siloAssetHourlySnapshots(
            first=10000, orderBy="timestamp", orderDirection="asc",
            where={"season_gte": 6074, "siloAsset": aid}
        )
        for aid in asset_ids 
    ]
    bs.SiloAssetHourlySnapshot.token_address = SyntheticField(
        lambda _id: _id.split("-")[1], 
        SyntheticField.STRING, 
        bs.SiloAssetHourlySnapshot.id, 
    )
    bs.SiloAssetHourlySnapshot.token_name = SyntheticField(
        lambda token_address: token_name_map[token_address], 
        SyntheticField.STRING, 
        bs.SiloAssetHourlySnapshot.token_address, 
    )
    bs.SiloAssetHourlySnapshot.total_deposited_amount = SyntheticField(
        lambda totalDepositedAmount, token_name: (
            totalDepositedAmount / DECIMALS_SILO_TOKENS[token_name]
        ), 
        SyntheticField.FLOAT, 
        [bs.SiloAssetHourlySnapshot.totalDepositedAmount, bs.SiloAssetHourlySnapshot.token_name],
    )
    bs.SiloAssetHourlySnapshot.total_deposited_bdv = SyntheticField(
        # TODO: Is this the correct multiplier? 
        lambda totalDepositedBDV: (
            totalDepositedBDV / DECIMALS_SILO_TOKENS['bean']
        ), 
        SyntheticField.FLOAT, 
        bs.SiloAssetHourlySnapshot.totalDepositedBDV,
    )
    df = pd.concat([
        sg.query_df(
            [
                q.season, 
                # q.timestamp, 
                q.token_address, 
                q.token_name, 
                q.total_deposited_amount, 
                q.total_deposited_bdv, 
            ],
            pagination_strategy=ShallowStrategy
        )
        for q in queries
    ])
    return df 

In [4]:

df_silo = query_silo_asset_hourly_snapshots(refresh=8).copy()
df_silo = remove_prefix(df_silo, 'siloAssetHourlySnapshots_')
df_silo = df_silo.groupby(['season', 'token_address']).last().reset_index() # TODO: verify if this does anything 
df_silo = df_silo.melt(
    id_vars=['season', 'token_address', 'token_name'], 
    value_vars=['total_deposited_amount', 'total_deposited_bdv']
)
"""
Hourly snapshots for a particular asset only exist when there was some update 
to amount of assets in the silo for that particular hour. 

Thus, we construct a full axis for all combinations of season, token, and metric 
then join our existing data onto this axis. We then forward fill values within 
groups of unique token and metric. 
"""
szn_min = df_silo.season.min()
szn_max = df_silo.season.max()
df_silo = pd.DataFrame([
    {
        "season": s, "token_name": token_name, 
        "token_address": token_addr, "variable": m
    }
    for s in range(szn_min, szn_max + 1)
        for token_name, token_addr in ADDRS_SILO_TOKENS.items()
            for m in ['total_deposited_amount', 'total_deposited_bdv']
]).merge(
    df_silo, how="left", on=["season", "token_address", "token_name", "variable"], validate="1:1"
)
df_silo = df_silo.sort_values("season").reset_index(drop=True)
# row for each combination of silo token (4) and metric (2)
validate_season_series(df_silo, count_expected=len(ADDRS_SILO_TOKENS) * 2)
# For reference: applying ffill within groups 
# https://www.pauldesalvo.com/how-to-apply-a-forward-fill-ffill-to-groups-in-pandas/
df_silo['value'] = df_silo.groupby(
    ['token_address', 'token_name', 'variable']
)['value'].transform(lambda x: x.ffill())
assert not df_silo.isna().values.any()

df_silo.head()
# ddf(df_silo)

,season,token_name,token_address,variable,value
0,6074,bean,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,total_deposited_amount,6.145891e+03
1,6074,bean,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,total_deposited_bdv,6.145891e+03
2,6074,bean_3crv,0xc9c32cd16bf7efb85ff14e0c8603cc90f6f2ee49,total_deposited_amount,9.857466e+04
3,6074,bean_3crv,0xc9c32cd16bf7efb85ff14e0c8603cc90f6f2ee49,total_deposited_bdv,9.517349e+04
4,6074,ur_bean,0x1bea0050e63e05fbb5d8ba2f10cf5800b6224449,total_deposited_amount,2.973191e+07


In [5]:
alt.Chart(df_silo.copy().loc[df_silo.variable == 'total_deposited_bdv']
).mark_area(
).encode(
    x=alt.X("season:Q"), 
    y=alt.Y("value:Q"), 
    color=alt.Color("token_name:N"), 
    tooltip=alt.Tooltip("value:Q")
).properties(width=700)

alt.Chart(...)

In [6]:
ddf(df_silo.copy().loc[(df_silo.season == szn_max) & (df_silo.variable == 'total_deposited_bdv')])
c = alt.Chart(df_silo.copy().loc[(df_silo.season == szn_max) & (df_silo.variable == 'total_deposited_bdv')]
).encode(
    theta=alt.Theta("value:Q", stack=True),
    color=alt.Color("token_name:N", legend=alt.Legend()),
    tooltip=alt.Tooltip("value:Q", format=",d")
).mark_arc(outerRadius=120, innerRadius=75
).properties(title="Silo Deposit by Token (BDV)")
c

,season,token_name,token_address,variable,value
9090,7210,bean,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,total_deposited_bdv,1639819.623
9092,7210,bean_3crv,0xc9c32cd16bf7efb85ff14e0c8603cc90f6f2ee49,total_deposited_bdv,565246.881
9094,7210,ur_bean,0x1bea0050e63e05fbb5d8ba2f10cf5800b6224449,total_deposited_bdv,11150636.693
9095,7210,ur_bean_3crv,0x1bea3ccd22f4ebd3d37d731ba31eeca95713716d,total_deposited_bdv,26883876.741


alt.Chart(...)

In [7]:
# query SeasonalDepositedUnripeBeans {
#   siloAssetDailySnapshots(
#     where: {
#       siloAsset: "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0x1bea0050e63e05fbb5d8ba2f10cf5800b6224449"
#     }
#     orderBy: season
#     orderDirection: desc
#     first: 10 
#   ) {
#     season
#     siloAsset {
#       totalDepositedAmount
#     	totalDepositedBDV
#     }
#   }
# }

In [8]:
output_chart(c)

<IPython.core.display.JSON object>